In [1]:
import glob
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import clima_anom as ca

import matplotlib.cbook
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import cartopy
import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

import sys
sys.path.append('../../function/')
from Funciones import dbz2mm, mm2dbz

In [2]:
with open ('../../json/directories.json') as f:
    DIRS = json.load(f)

In [3]:
file_list = pd.read_csv(DIRS['dir_ENSEMBLE_list']+'TRMM_GPM_summer_200012_2021_02.csv')
len_files = len(file_list)
print(f'Files Number: {len_files}')
print(file_list['file_name'].iloc[0])
print(file_list['file_name'].iloc[-1])

Files Number: 63
/mnt/Data/Data/PF/TRMM/rpf_SA/rpf_200012_level2.csv
/mnt/Data/Data/PF/GPM/rgpf_SA/rgpf_202102_level2.csv


In [4]:
Comunidad_file = 'Comm_IMERG_05x05_sync_summer_P90_tau3_SP95'

In [5]:
levels = np.arange(0.5,20.5,0.5)

for nn in range(63):

    data = pd.read_csv(file_list['file_name'][nn], sep=',')
    data[data==-88.88] = np.nan
    data[data==-99.99] = np.nan

    PF_GPD = gpd.GeoDataFrame(data[data.columns[2:]], geometry=gpd.points_from_xy(data['lon'],data['lat'], crs="EPSG:4326"))

    comm_GPD = gpd.read_file('/mnt/Data/Github/Complex-Network-AS/mapping/communities/'+Comunidad_file+'.shp')

    num_comm = len(comm_GPD['geometry'])
    num_points = len(PF_GPD['geometry']) 

    points_interception = []  

    for t in range(num_points):

        if comm_GPD['geometry'][0].contains(PF_GPD['geometry'][t]) == True:
            points_interception.append(t)

    dic_tmp = data.loc[points_interception]
    dic_tmp = dic_tmp.reset_index(drop=True)

    if nn == 0:
        dict_out = dic_tmp

    else:
        dict_out = pd.concat([dic_tmp, dict_out], axis=0)

    print(f'Points in the community: {len(dic_tmp)}')

# export dataframe to csv
salida = '/mnt/Data/Data/PF/ENSEMBLE/ENSEMBLE_SA/'+Comunidad_file+'.csv'
print(f'Exporting data to {salida}')
dict_out.to_csv(salida,header=True,index=False)

Points in the community: 10219
Points in the community: 10951
Points in the community: 9795
Points in the community: 8838
Points in the community: 8606
Points in the community: 10421
Points in the community: 10092
Points in the community: 9219
Points in the community: 8850
Points in the community: 7860
Points in the community: 9432
Points in the community: 10873
Points in the community: 8857
Points in the community: 9046
Points in the community: 9280
Points in the community: 10121
Points in the community: 9180
Points in the community: 10715
Points in the community: 9223
Points in the community: 10745
Points in the community: 7855
Points in the community: 9847
Points in the community: 10671
Points in the community: 9365
Points in the community: 9556
Points in the community: 10123
Points in the community: 10394
Points in the community: 9799
Points in the community: 9886
Points in the community: 8142
Points in the community: 7675
Points in the community: 11212
Points in the community: 940